In [ ]:
# 📚 Imports
import pandas as pd
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D,                          Reshape, Bidirectional, LSTM, Dense, Dropout,                          Attention, GlobalAveragePooling1D, Flatten
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import optuna
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

# 📦 Dataset Paths
base_dir = "/kaggle/input/music-classification-wav/IA"
audio_dir = os.path.join(base_dir, "TRAIN_V2", "data_out_2")
train_csv = os.path.join(base_dir, "train.csv")
genres_csv = os.path.join(base_dir, "genres.csv")

# 📥 Load Labels
train_df = pd.read_csv(train_csv)
genres_df = pd.read_csv(genres_csv)

# 🎵 Feature Extraction Function
def extract_features(filepath, sr=22050):
    y, sr = librosa.load(filepath, sr=sr, duration=30)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
    features = np.hstack([
        np.mean(mfcc, axis=1),
        np.mean(chroma, axis=1),
        np.mean(mel, axis=1)[:20],
        np.mean(contrast, axis=1),
        np.mean(tonnetz, axis=1)
    ])
    return features

# 🧠 Extract Features and Labels
X, y = [], []
for i, row in train_df.iterrows():
    genre_id = row['label']
    file = row['filename']
    genre_folder = os.path.join(audio_dir, str(genre_id))
    path = os.path.join(genre_folder, file)
    try:
        features = extract_features(path)
        X.append(features)
        y.append(genre_id)
    except Exception as e:
        print(f"Error processing {path}: {e}")

X = np.array(X)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse_output=False)
y_onehot = onehot_encoder.fit_transform(y_encoded.reshape(-1, 1))

X = np.expand_dims(X, -1)
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)
